# Modelling Pipeline

## 1. Import Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer, FunctionTransformer, MinMaxScaler, Normalizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, accuracy_score
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from scipy.stats import shapiro
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import classification_report,  confusion_matrix
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## 2. Prepare Data

### 2.1 Load Data

In [2]:
df = pd.read_csv('clean data/final_data.csv', index_col=0)
df.head()

,Country,WS_MDG,WUE_SDG,WS_SDG,Temp,Rain,IRWR,ERWR,TRWR,Dep_ratio,rural_pop,urban_pop,HDI,r_u,r_u_access,pop_growth,mort_rate,GDP_pcp,life_ex
0,AFG,31.045462,0.923778,54.757019,14.074742,349.736945,47.15,18.18,65.33,0.278280,26558.609,8971.472,0.493,2.960340,0.601023,3.06,53.2,2226.0,63.4
1,AGO,0.475539,142.467836,1.871883,22.182196,960.024065,148.00,0.40,148.40,0.002695,10472.554,19311.639,0.576,0.542292,0.374005,3.44,58.6,7859.4,59.2
2,ALB,3.933775,6.656907,7.139423,12.754647,1079.459167,26.90,3.30,30.20,0.109272,1190.155,1740.032,0.789,0.683985,1.003161,-0.20,8.6,12227.4,78.0
3,ARE,1708.000000,92.773763,1708.000000,28.010773,64.449765,0.15,0.00,0.15,0.000000,1292.709,8107.436,0.864,0.159447,1.004016,0.74,7.0,64243.0,77.2
4,ARG,4.301333,13.616564,10.456664,14.767043,598.510300,292.00,584.24,876.24,0.666758,3652.804,40618.237,0.832,0.089930,1.010101,1.08,10.2,23732.2,76.0


### 2.1 Add additional climate variables

In [3]:
df['IRWR_capita'] = df['IRWR'] / ((df['urban_pop'] + df['rural_pop']) * 1000)
df['ERWR_capita'] = df['ERWR'] / ((df['urban_pop'] + df['rural_pop']) * 1000)
df['TRWR_capita'] = df['TRWR'] / ((df['urban_pop'] + df['rural_pop']) * 1000)

### 2.2 Split dataframe into chosen predictor and target variables

In [4]:
df_pred_climate = df.iloc[:, np.r_[4:10, 19:22]]
df_pred_socioec = df.iloc[:, 10:18]
df_pred = df.iloc[:, 4:22]
df_target = df.iloc[:, 1:4]

## 3. Pipeline: Predictor Variables

### 3.1 Scalers

In [5]:
def log_transform(x):
    return np.log(x + 1)

In [6]:
logscaler = FunctionTransformer(log_transform)

### 3.2 Dimensionality reduction

In [7]:
pca_target = PCA(n_components=2)

### 3.3 Setup pipeline

In [8]:
pipe_target = Pipeline([
    ('scaler', logscaler),
    ('reduce_dim', pca_target)
])

### 3.4 Fit pipeline

In [9]:
df_target_ = pipe_target.fit_transform(df_target)

In [10]:
print('Principale components: \n', pipe_target.steps[1][1].components_)

Principale components: 
 [[ 0.7208814  -0.06902087  0.68961303]
 [ 0.06914256  0.99722682  0.0275312 ]]


Principal component 1 = Water stress\
Principal component 2 = Water use efficiency

In [11]:
print('Explained variance: \n',
      pipe_target.steps[1][1].explained_variance_ratio_)

Explained variance: 
 [0.75740679 0.2363991 ]


## 6. Pipeline: Climate + Socio-Economic Variables

### 6.1 Scalers

All scalers combinations to test:

In [22]:
scalers_to_test = [
    ColumnTransformer(remainder='passthrough',
                      transformers=[
                          ("logscaler", logscaler, [
                           'Rain', 'IRWR', 'ERWR', 'TRWR', 'IRWR_capita', 'ERWR_capita', 'TRWR_capita','rural_pop', 'urban_pop', 'GDP_pcp'])
                      ]),
    ColumnTransformer(remainder='passthrough',
                      transformers=[
                          ("logscaler", logscaler, [
                           'Rain', 'IRWR', 'ERWR', 'TRWR', 'IRWR_capita', 'ERWR_capita', 'TRWR_capita','rural_pop', 'urban_pop', 'GDP_pcp']),
                          ("standardscaler", StandardScaler(),
                           df_pred.columns.values)
                      ]),
    ColumnTransformer(remainder='passthrough',
                      transformers=[
                          ("logscaler", logscaler, [
                           'Rain', 'IRWR', 'ERWR', 'TRWR', 'IRWR_capita', 'ERWR_capita', 'TRWR_capita','rural_pop', 'urban_pop', 'GDP_pcp']),
                          ("robustscaler", RobustScaler(),
                           df_pred.columns.values)
                      ]),
    ColumnTransformer(remainder='passthrough',
                      transformers=[
                          ("logscaler", logscaler, [
                           'Rain', 'IRWR', 'ERWR', 'TRWR', 'IRWR_capita', 'ERWR_capita', 'TRWR_capita','rural_pop', 'urban_pop', 'GDP_pcp']),
                          ("minmaxscaler", MinMaxScaler(),
                           df_pred.columns.values)
                      ])
]

### 6.2 Dimensionality reduction

All number of components to test for the PCA analysis: 

In [23]:
pca_pred = PCA()

In [24]:
n_components_to_test = np.arange(3, 15)

### 6.3 Regression model

All models + model parameters to test: 

In [25]:
model_1 = RandomForestRegressor(random_state=0)
max_depth_to_test = np.arange(2, 15)

model_2 = LinearRegression()

model_3 = Ridge()
alpha_to_test = np.arange(1, 15)

### 6.4 List of parameters to test

Make list of parameter dictionaries (one for each model):

In [26]:
params = [
    {'scaler': scalers_to_test,
     'reduce_dim__n_components': n_components_to_test,
     'regressor': [model_2]},

    {'scaler': scalers_to_test,
     'reduce_dim__n_components': n_components_to_test,
     'regressor': [model_3],
     'regressor__alpha': alpha_to_test}
]

### 6.5 Train, test, split + setup pipeline

In [101]:
X_train, X_test, y_train, y_test = train_test_split(
    df_pred, df_target, random_state=1)

In [102]:
pipe_pred = Pipeline([
    ('scaler', scalers_to_test[0]),
    ('reduce_dim', pca_pred),
    ('regressor', model_1)
])

In [103]:
y_train

,WS_MDG,WUE_SDG,WS_SDG
42,0.297373,40.949707,0.297373
100,1.382707,31.517072,6.089404
55,33.884557,2.820583,66.492093
53,4.325962,23.880203,7.770294
44,0.083795,94.789871,0.502166
...,...,...,...
9,21.825137,106.071515,49.066339
72,0.518317,132.417395,1.067177
12,2.923314,5.715195,5.723339
107,1.364943,194.076472,3.427128


### 6.6 Gridsearch pipeline untransformed target variables

#### 6.6.1 Target variable 1: WS_MDG

In [104]:
gridsearch_1 = GridSearchCV(
    pipe_pred, params, verbose=1).fit(X_train, y_train['WS_MDG'])

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


In [105]:
gridsearch_1.best_params_

{'reduce_dim__n_components': 3,
 'regressor': Ridge(alpha=14),
 'regressor__alpha': 14,
 'scaler': ColumnTransformer(remainder='passthrough',
                   transformers=[('logscaler',
                                  FunctionTransformer(func=<function log_transform at 0x000001FA1DECC268>),
                                  ['Rain', 'IRWR', 'ERWR', 'TRWR', 'IRWR_capita',
                                   'ERWR_capita', 'TRWR_capita', 'rural_pop',
                                   'urban_pop', 'GDP_pcp']),
                                 ('robustscaler', RobustScaler(),
                                  array(['Temp', 'Rain', 'IRWR', 'ERWR', 'TRWR', 'Dep_ratio', 'rural_pop',
        'urban_pop', 'HDI', 'r_u', 'r_u_access', 'pop_growth', 'mort_rate',
        'GDP_pcp', 'life_ex', 'IRWR_capita', 'ERWR_capita', 'TRWR_capita'],
       dtype=object))])}

#### 6.6.2 Target variable 2: WUE_SDG

In [106]:
gridsearch_2 = GridSearchCV(
    pipe_pred, params, verbose=1).fit(X_train, y_train['WUE_SDG'])

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


In [107]:
gridsearch_2.best_params_

{'reduce_dim__n_components': 3,
 'regressor': Ridge(alpha=14),
 'regressor__alpha': 14,
 'scaler': ColumnTransformer(remainder='passthrough',
                   transformers=[('logscaler',
                                  FunctionTransformer(func=<function log_transform at 0x000001FA1DECC268>),
                                  ['Rain', 'IRWR', 'ERWR', 'TRWR', 'IRWR_capita',
                                   'ERWR_capita', 'TRWR_capita', 'rural_pop',
                                   'urban_pop', 'GDP_pcp']),
                                 ('robustscaler', RobustScaler(),
                                  array(['Temp', 'Rain', 'IRWR', 'ERWR', 'TRWR', 'Dep_ratio', 'rural_pop',
        'urban_pop', 'HDI', 'r_u', 'r_u_access', 'pop_growth', 'mort_rate',
        'GDP_pcp', 'life_ex', 'IRWR_capita', 'ERWR_capita', 'TRWR_capita'],
       dtype=object))])}

#### 6.6.3 Target variable 3: WS_SDG

In [108]:
gridsearch_3 = GridSearchCV(
    pipe_pred, params, verbose=1).fit(X_train, y_train['WS_SDG'])

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


In [109]:
gridsearch_3.best_params_

{'reduce_dim__n_components': 3,
 'regressor': Ridge(alpha=14),
 'regressor__alpha': 14,
 'scaler': ColumnTransformer(remainder='passthrough',
                   transformers=[('logscaler',
                                  FunctionTransformer(func=<function log_transform at 0x000001FA1DECC268>),
                                  ['Rain', 'IRWR', 'ERWR', 'TRWR', 'IRWR_capita',
                                   'ERWR_capita', 'TRWR_capita', 'rural_pop',
                                   'urban_pop', 'GDP_pcp']),
                                 ('robustscaler', RobustScaler(),
                                  array(['Temp', 'Rain', 'IRWR', 'ERWR', 'TRWR', 'Dep_ratio', 'rural_pop',
        'urban_pop', 'HDI', 'r_u', 'r_u_access', 'pop_growth', 'mort_rate',
        'GDP_pcp', 'life_ex', 'IRWR_capita', 'ERWR_capita', 'TRWR_capita'],
       dtype=object))])}

### 6.7 Best fit model untransformed target variables

#### 6.7.1 Target variable 1: WS_MDG

In [40]:
pipe_pred_1 = Pipeline([
    ('scaler', scalers_to_test[0]),
    ('reduce_dim', PCA(n_components=3)),
    ('regressor', Ridge(alpha=14))
])

In [44]:
pipe_pred_1.fit(X_train, y_train['WS_MDG'])
print('Model score: ', pipe_pred_1.score(X_test, y_test['WS_MDG']))
y_pred = pipe_pred_1.predict(df_pred)
print('R²', r2_score(df_target['WS_MDG'], y_pred))

Model score:  0.07716704720898693
R² 0.10421997376108483


#### 6.7.2 Target variable 2: WUE_SDG

In [45]:
pipe_pred_2 = Pipeline([
    ('scaler', scalers_to_test[0]),
    ('reduce_dim', PCA(n_components=14)),
    ('regressor', Ridge(alpha=4))
])

In [48]:
pipe_pred_2.fit(X_train, y_train['WUE_SDG'])
print('Model score: ', pipe_pred_2.score(X_test, y_test['WUE_SDG']))
y_pred = pipe_pred_2.predict(df_pred)
print('R²', r2_score(df_target['WUE_SDG'], y_pred))

Model score:  0.15977232963766086
R² 0.2338670654381797


#### 6.7.2 Target variable 3: WS_SDG

In [51]:
pipe_pred_3 = Pipeline([
    ('scaler', scalers_to_test[0]),
    ('reduce_dim', PCA(n_components=14)),
    ('regressor', Ridge(alpha=14))
])

In [56]:
pipe_pred_3.fit(X_train, y_train['WS_SDG'])
print('Model score: ', pipe_pred_3.score(X_test, y_test['WS_SDG']))
y_pred = pipe_pred_3.predict(df_pred)
print('R²', r2_score(df_target['WS_SDG'], y_pred))

Model score:  0.14172051000433428
R² 0.21094794236279968


# Gridsearch back on PCA tf target vars

In [82]:
X_train, X_test, y_train, y_test = train_test_split(
    df_pred, df_target_, random_state=0)

In [83]:
y_train[1]

array([-3.10366648,  0.88189786])

#### 6.7.1 Target principal component 1: water stress

In [84]:
gridsearch_1 = GridSearchCV(
    pipe_pred, params, verbose=1).fit(X_train, y_train[:, 0])
gridsearch_1.best_params_

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


{'reduce_dim__n_components': 12,
 'regressor': Ridge(alpha=4),
 'regressor__alpha': 4,
 'scaler': ColumnTransformer(remainder='passthrough',
                   transformers=[('logscaler',
                                  FunctionTransformer(func=<function log_transform at 0x000001FA1DECC268>),
                                  ['Rain', 'IRWR', 'ERWR', 'TRWR', 'IRWR_capita',
                                   'ERWR_capita', 'TRWR_capita', 'rural_pop',
                                   'urban_pop', 'GDP_pcp'])])}

In [85]:
pipe_pred_1 = Pipeline([
    ('scaler', scalers_to_test[0]),
    ('reduce_dim', PCA(n_components=12)),
    ('regressor', Ridge(alpha=4))
])

In [86]:
pipe_pred_1.fit(X_train, y_train[:, 0])
print('Model score: ', pipe_pred_1.score(X_test, y_test[:, 0]))
y_pred = pipe_pred_1.predict(df_pred)
print('R²', r2_score(df_target_[:, 0], y_pred))

Model score:  0.6384512159002251
R² 0.7582585643406689


In [136]:
for i in range(1,10):
    print(i)
    X_train, X_test, y_train, y_test = train_test_split(
        df_pred, df_target_)
    print(y_train[1])
    pipe_pred_1.fit(X_train, y_train[:, 0])
    print('Model score: ', pipe_pred_1.score(X_test, y_test[:, 0]))
    y_pred = pipe_pred_1.predict(X_test)
    y_pred_train = pipe_pred_4.predict(X_train)
    print('R² train', r2_score( y_pred_train,y_train[:,0]))
    print('R²', r2_score(y_test[:, 0], y_pred))

1
[-0.03446617 -0.85033001]
Model score:  0.6852020939092209
R² train -0.06069254326210349
R² 0.6852020939092209
2
[-0.17791501 -0.94159024]
Model score:  0.7532182053221645
R² train -0.07276329565089523
R² 0.7532182053221645
3
[-1.42823185  0.37598225]
Model score:  0.6926897589196481
R² train -0.05465228777504039
R² 0.6926897589196481
4
[-0.155684   1.7139551]
Model score:  0.8022095359803515
R² train -0.033427825962511815
R² 0.8022095359803515
5
[-1.88199048  2.14915967]
Model score:  0.8170541323026932
R² train -0.04309733516815184
R² 0.8170541323026932
6
[-1.0380791   1.22733331]
Model score:  0.6494650354733883
R² train -0.044972181960070357
R² 0.6494650354733883
7
[1.46645506 1.77446193]
Model score:  0.7997538904242238
R² train -0.04256020573412034
R² 0.7997538904242238
8
[ 0.77067348 -1.29051268]
Model score:  0.7224852373081552
R² train -0.06625796596304845
R² 0.7224852373081552
9
[ 2.16495556 -1.40125756]
Model score:  0.655458568174849
R² train -0.07055068086054561
R² 0.655

In [88]:
len(X_test)

31

In [89]:
len(X_train)

91

In [122]:
pipe_pred_4 = Pipeline([
    ('scaler', scalers_to_test[0]),
    ('reduce_dim', PCA(n_components=12)),
    ('regressor', Ridge(alpha=6))
])

In [139]:
for i in range(1,10):
    print(i)
    X_train, X_test, y_train, y_test = train_test_split(
        df_pred, df_target)
    print(y_train.head(1))
    pipe_pred_4.fit(X_train, y_train['WS_SDG'])
    print('Model score: ', pipe_pred_4.score(X_test, y_test['WS_SDG']))
    y_pred_train = pipe_pred_4.predict(X_train)
    print('R² train', r2_score(y_train['WS_SDG'], y_pred_train))
    y_pred = pipe_pred_4.predict(X_test)
    print('R² test', r2_score( y_pred,y_test['WS_SDG']))

1
       WS_MDG   WUE_SDG     WS_SDG
54  11.028632  3.745191  29.696545
Model score:  0.05437498634360938
R² train 0.6238184293133968
R² test 0.12508046739783163
2
      WS_MDG    WUE_SDG    WS_SDG
28  0.576352  19.030056  2.036213
Model score:  0.12773466576601022
R² train 0.6759203987778388
R² test -1.0290121749612737
3
      WS_MDG   WUE_SDG     WS_SDG
119  9.25915  2.349448  18.130315
Model score:  -0.027960863105741307
R² train 0.6858470733982511
R² test -0.02839836368062021
4
      WS_MDG     WUE_SDG     WS_SDG
9  21.825137  106.071515  49.066339
Model score:  0.1627253873913066
R² train 0.44521637666784464
R² test -24.12942580396095
5
      WS_MDG    WUE_SDG    WS_SDG
85  0.706339  35.203598  0.861553
Model score:  0.15912593895161076
R² train 0.5009872559707476
R² test -19.701938763300703
6
       WS_MDG  WUE_SDG     WS_SDG
57  67.824437  4.77385  81.289081
Model score:  -0.04983133202690482
R² train 0.6330750805638703
R² test 0.11943366710458914
7
       WS_MDG     WUE_SDG    

In [133]:
r2_score(y_train[:,0], y_pred_train)

-8669.2602420022

In [134]:
y_pred_train

array([ 5.17046529e+02, -2.08771342e+02,  9.91113279e+01,  5.52587498e+01,
       -1.22486276e+02, -7.06040137e+01, -3.91956274e+01,  3.73952600e+02,
       -3.23126407e+02, -1.13942397e+02, -2.68879134e+01,  7.38812653e+01,
       -4.75135592e+01,  1.02303044e+03, -1.03466352e+02,  1.42802016e+02,
        2.45381175e+01, -1.33616396e+02,  8.68564508e+01,  1.44896091e+02,
        8.28662194e+01,  4.06232484e+02,  5.64621766e+01,  1.62331784e+01,
        1.02440993e+02, -9.81393303e-01, -1.63677747e+02, -1.45538190e+01,
        2.90905807e+01,  1.23203961e+02,  2.12325760e+01, -6.55642918e+01,
       -1.27449870e+02,  1.70914555e+02,  3.42899396e+02,  4.06443193e+02,
        1.64408220e+01, -7.39742464e+01,  6.57805185e+02, -1.65308587e+02,
        2.70255123e+01,  1.60560257e+02,  1.34361882e+02, -6.77520994e+01,
        2.73834239e+02,  1.00396593e+02,  1.04684443e+02, -1.03625344e+02,
        6.87619752e+01, -9.96202369e+01, -7.82813640e+01,  1.41346927e+02,
       -7.21844285e+01,  

In [135]:
y_train

array([[ 7.02133997e+00,  2.03781716e+00],
       [-2.35608590e+00,  1.70892582e+00],
       [ 3.41461622e+00, -1.07420698e+00],
       [-1.10864021e+00, -2.18307732e-01],
       [-9.82276030e-01,  2.11201401e-01],
       [-1.52701391e-01, -9.46268678e-01],
       [-1.55228523e+00,  2.66619690e-01],
       [ 2.76804829e+00,  2.31697240e+00],
       [-1.86249862e+00,  2.73817933e+00],
       [ 1.45818735e+00,  1.89379685e-01],
       [-3.44661658e-02, -8.50330011e-01],
       [-2.59848097e+00,  4.23177450e-01],
       [-8.86658804e-01,  4.72183954e-01],
       [ 5.03649247e+00,  2.62311969e+00],
       [-3.06200186e-01, -2.47507130e+00],
       [-2.01588732e+00,  1.15568864e+00],
       [ 2.01404117e+00, -6.12678651e-01],
       [-2.60322936e+00,  1.09180778e+00],
       [-1.82199648e+00, -9.75035756e-01],
       [ 1.66608849e+00, -6.93982488e-01],
       [ 8.87969828e-01, -1.48818601e-01],
       [ 2.75152025e+00,  2.03783043e+00],
       [-1.66276372e+00, -4.45331603e-01],
       [-1.